In [1]:
import ee
import geemap
import pandas as pd
import pyproj
from shapely.geometry import Point, mapping
from shapely.ops import transform
from shapely.geometry import shape
from functools import partial

import warnings
warnings.filterwarnings('ignore')

In [48]:
# Create a map centered at (lat, lon).
Map = geemap.Map()

In [49]:
palette = ['ffd300', 'ff2626', '00a8e2', 'ff9e0a', '267000', 'ffff00',
       '70a500', '00af49', 'dda50a', 'dda50a', '7cd3ff', 'e2007c',
       '896054', 'd8b56b', 'a57000', 'd69ebc', '707000', 'aa007c',
       'a05989', '700049', 'd69ebc', 'd1ff00', '7c99ff', 'd6d600',
       'd1ff00', '00af49', 'ffa5e2', 'a5f28c', '00af49', 'd69ebc',
       'a800e2', 'a50000', '702600', '00af49', 'af7cff', '702600',
       'ff6666', 'ff6666', 'ffcc66', 'ff6666', '00af49', '00ddaf',
       '54ff00', 'f2a377', 'ff6666', '00af49', '7cd3ff', 'e8bfff',
       'afffdd', '00af49', 'bfbf77', '93cc93', 'c6d69e', 'ccbfa3',
       'ff00ff', 'ff8eaa', 'ba004f', '704489', '007777', 'af9970',
       'ffff7c', 'b5705b', '00a582', 'e8d6af', 'af9970', 'f2f2f2',
       '999999', '4970a3', '7cafaf', 'e8ffbf', '00ffff', '4970a3',
       'd3e2f9', '999999', '999999', '999999', '999999', 'ccbfa3',
       '93cc93', '93cc93', '93cc93', 'c6d69e', 'e8ffbf', '7cafaf',
       '7cafaf', '00ff8c', 'd69ebc', 'ff6666', 'ff6666', 'ff6666',
       'ff6666', 'ff8eaa', '334933', 'e27026', 'ff6666', 'ff6666',
       '739755', 'ff6666', 'af9970', 'ff8eaa', 'ff6666', 'ff8eaa',
       'ff6666', 'ff6666', 'ff8eaa', '00af49', 'ffd300', 'ffd300',
       'ff6666', 'f8d248', 'ff6666', '896054', 'ff6666', 'ff2626',
       'e2007c', 'ff9e0a', 'ff9e0a', 'a57000', 'ffd300', 'a57000',
       '267000', '267000', 'ffd300', '000099', 'ff6666', 'ff6666',
       'ff6666', 'ff6666', 'ff6666', 'ff6666', 'ff6666', 'ff6666',
       '267000']

In [50]:
## https://gis.stackexchange.com/questions/345642/export-landcover-area-after-classification-google-earth-engine

In [51]:

def reproject_geom(geom, src_epsg, dst_epsg):
    """Reproject a shapely geometry given a source EPSG and a
    target EPSG.
    """
    src_proj = pyproj.Proj(init='epsg:{}'.format(src_epsg))
    dst_proj = pyproj.Proj(init='epsg:{}'.format(dst_epsg))
    reproj = partial(pyproj.transform, src_proj, dst_proj)
    return transform(reproj, geom)


def buffer_extent(lat, lon, dst_epsg, buffer_size):
    """Generate a buffer around a location and returns
    the geometry corresponding to its spatial envelope.
    """
    center = reproject_geom(
        Point(lon, lat), src_epsg=4326, dst_epsg=dst_epsg)
    buffer = center.buffer(buffer_size)
    return buffer.exterior.envelope


def geojson_crs(epsg):
    """Generate a GeoJSON CRS member from an EPSG code."""
    epsg = int(epsg)
    if epsg == 4326:
        coordinate_order = [1, 0]
    else:
        coordinate_order = [0, 1]
    return {
        'type': 'EPSG',
        'properties': {'code': epsg, 'coordinate_order': coordinate_order}}


def as_geojson(geom, epsg=None):
    """Get a GeoJSON-like dictionnary of a single shapely geometry."""
    geojson = {'type': 'Feature', 'geometry': mapping(geom)}
    if epsg:
        geojson.update(crs=geojson_crs(epsg))
    return geojson

In [52]:
lat = 40.71
lon = -100.55

extent = buffer_extent(lat, lon, 3857 , buffer_size=5000) #1.6594, 28.0339
extent = as_geojson(extent,3857)
aoi = reproject_geom(shape(extent['geometry']), 3857, 4326)
xmin, ymin, xmax, ymax = [round(coord, 3) for coord in aoi.bounds]

bbox = [xmin, ymin, xmax, ymax]
gg = ee.Geometry.Rectangle(bbox)

### working code crop land

In [53]:
Map = geemap.Map()
# Map = geemap.Map(center=[40.71, -100.55], zoom=4)

point = ee.Geometry.Point([-100.55, 40.71])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2016-01-01', '2016-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(center=[40.71, -100.55000000000001], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [54]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()


'2016-04-22'

In [55]:
image.get('CLOUD_COVER').getInfo()


0

In [56]:
geometry = ee.Geometry.Point([-100.55, 40.71])


In [57]:
cropLandcover = ee.ImageCollection('USDA/NASS/CDL')\
                .filterBounds(geometry) \
                .filter(ee.Filter.date('2018-01-01', '2019-12-31')).first().select('cultivated')
                #.filter(ee.Filter.date('2018-01-01', '2019-12-31')).first().select('cultivated') #cropland
                



landcoverPalette = ['ffd300', 'ff2626', '00a8e2', 'ff9e0a', '267000', 'ffff00',
       '70a500', '00af49', 'dda50a', 'dda50a', '7cd3ff', 'e2007c',
       '896054', 'd8b56b', 'a57000', 'd69ebc', '707000', 'aa007c',
       'a05989', '700049', 'd69ebc', 'd1ff00', '7c99ff', 'd6d600',
       'd1ff00', '00af49', 'ffa5e2', 'a5f28c', '00af49', 'd69ebc',
       'a800e2', 'a50000', '702600', '00af49', 'af7cff', '702600',
       'ff6666', 'ff6666', 'ffcc66', 'ff6666', '00af49', '00ddaf',
       '54ff00', 'f2a377', 'ff6666', '00af49', '7cd3ff', 'e8bfff',
       'afffdd', '00af49', 'bfbf77', '93cc93', 'c6d69e', 'ccbfa3',
       'ff00ff', 'ff8eaa', 'ba004f', '704489', '007777', 'af9970',
       'ffff7c', 'b5705b', '00a582', 'e8d6af', 'af9970', 'f2f2f2',
       '999999', '4970a3', '7cafaf', 'e8ffbf', '00ffff', '4970a3',
       'd3e2f9', '999999', '999999', '999999', '999999', 'ccbfa3',
       '93cc93', '93cc93', '93cc93', 'c6d69e', 'e8ffbf', '7cafaf',
       '7cafaf', '00ff8c', 'd69ebc', 'ff6666', 'ff6666', 'ff6666',
       'ff6666', 'ff8eaa', '334933', 'e27026', 'ff6666', 'ff6666',
       '739755', 'ff6666', 'af9970', 'ff8eaa', 'ff6666', 'ff8eaa',
       'ff6666', 'ff6666', 'ff8eaa', '00af49', 'ffd300', 'ffd300',
       'ff6666', 'f8d248', 'ff6666', '896054', 'ff6666', 'ff2626',
       'e2007c', 'ff9e0a', 'ff9e0a', 'a57000', 'ffd300', 'a57000',
       '267000', '267000', 'ffd300', '000099', 'ff6666', 'ff6666',
       'ff6666', 'ff6666', 'ff6666', 'ff6666', 'ff6666', 'ff6666',
       '267000']

landcoverVisualization = {'palette': landcoverPalette, 'min': 1, 'max': 254, 'format': 'png'}
# Map.addLayer(cropLandcover,
#              landcoverVisualization,
#              'Crop Landcover')

In [58]:
Map.addLayer(cropLandcover,{}, 'cultivated')
Map

Map(bottom=24941.0, center=[40.71, -100.55000000000001], controls=(WidgetControl(options=['position'], widget=…

In [59]:

# Make the training dataset.
points = cropLandcover.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [60]:
print(points.size().getInfo())


5000


In [61]:
print(points.first().getInfo())


{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-99.58353038155882, 39.97737372431678]}, 'id': '0', 'properties': {'cultivated': 2}}


In [62]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'cultivated' #cultivated cropland

# Overlay the points on the imagery to get training.
# training = image.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })
sample = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})
sample = sample.randomColumn()

split = 0.8

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))


In [63]:
training.first().getInfo()


{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 576,
  'B2': 749,
  'B3': 1076,
  'B4': 1414,
  'B5': 2352,
  'B6': 3345,
  'B7': 2499,
  'cultivated': 2,
  'random': 0.781419414845358}}

In [64]:
validation.first().getInfo()


{'type': 'Feature',
 'geometry': None,
 'id': '13_0',
 'properties': {'B1': 437,
  'B2': 554,
  'B3': 750,
  'B4': 936,
  'B5': 1658,
  'B6': 2456,
  'B7': 1927,
  'cultivated': 2,
  'random': 0.8577752970044478}}

In [65]:
# trained = ee.Classifier.smileCart().train(training, label, bands)
classifier = ee.Classifier.smileRandomForest(500).train(training, label, bands)


In [66]:

# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier) ## trained

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
#Map

In [67]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
#Map

In [68]:
class_values = cropLandcover.get('cultivated_class_values').getInfo()

In [391]:
#class_values = cropLandcover.get('cropland_class_values').getInfo()

In [69]:
n_classes = len(class_values)
new_class_values = list(range(0, n_classes))


In [70]:
class_palette = cropLandcover.get('cultivated_class_palette').getInfo()
#class_palette

In [393]:
#class_palette = cropLandcover.get('cropland_class_palette').getInfo()
#class_palette

In [71]:
croplandcover = result.set('classification_class_values', class_values)
croplandcover = croplandcover.set('classification_class_palette', class_palette)

In [72]:
Map.addLayer(croplandcover, {}, 'Crop Landcover')
Map

Map(bottom=24941.0, center=[40.71, -100.55000000000001], controls=(WidgetControl(options=['position'], widget=…

In [300]:
# cropLandcover.getInfo()

In [73]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [74]:
#Map.add_legend(builtin_legend='USDA/NASS/CDL')
# Map

In [75]:
train_accuracy = classifier.confusionMatrix()


In [76]:
# train_accuracy.getInfo()


In [77]:
train_accuracy.accuracy().getInfo()


0.9987493746873437

In [78]:
train_accuracy.kappa().getInfo()


0.9974987418634036

In [79]:
# print(train_accuracy.getInfo())

In [80]:
# train_accuracy.consumersAccuracy().getInfo()


In [81]:
validated = validation.classify(classifier)


In [82]:
validated.first().getInfo()


{'type': 'Feature',
 'geometry': None,
 'id': '13_0',
 'properties': {'B1': 437,
  'B2': 554,
  'B3': 750,
  'B4': 936,
  'B5': 1658,
  'B6': 2456,
  'B7': 1927,
  'classification': 2,
  'cultivated': 2,
  'random': 0.8577752970044478}}

In [96]:
test_accuracy = validated.errorMatrix('cultivated', 'classification')


In [97]:
#test_accuracy = validated.errorMatrix('croplandcover', 'classification')


### test accuracy

In [98]:
test_accuracy.accuracy().getInfo()


0.899

In [99]:
test_accuracy.kappa().getInfo()


0.797739488461138

In [100]:
test_accuracy.producersAccuracy().getInfo()


[[0], [0.9105058365758755], [0.8868312757201646]]

In [101]:
test_accuracy.consumersAccuracy().getInfo()


[[0, 0.8948374760994264, 0.9035639412997903]]

In [102]:
croplandcover = croplandcover.remap(class_values, class_values).select(['remapped'], ['classification'])


In [103]:
cropLandcover = cropLandcover.set('classification_class_values', class_values)
cropLandcover = cropLandcover.set('classification_class_palette', landcoverPalette)

In [104]:
Map.addLayer(cropLandcover, {}, 'Final land cover')
Map

Map(bottom=6586.0, center=[38.58252615935333, -104.677734375], controls=(WidgetControl(options=['position'], w…

In [105]:
# import os
# out_dir = "D:/env_geemap/dataImages/"
# out_file = os.path.join(out_dir, 'croplandcoverfFinal.tif')

In [106]:
# geemap.ee_export_image(croplandcover, filename=out_file, scale=900,)
